# This is a scrip to compress .jpeg files

This script should be run in the folder where the original uncompressed files are. 
The name of the folder will be used to rename the compressed files in sequential order. 
Two new folders will be created: 
- `\kofi` contains compressed photos with the same resolution as the original photo if size is above > 24.8 mb; otherwise it will just copy and rename following project name 
- `\socials` contains photos with lower resolution set by the variable  `img_max_dim` 

In the project folder will be added a .csv file with a summary of the compression applyed to each photo, if any.

In [ ]:
# import libraries
from PIL import Image
import os
import shutil
import csv
import pandas as pd

## set constants
Image.MAX_IMAGE_PIXELS = 1000000000  # set the limit to 1 billion pixels to support large files 

## set user defined variables
year = 2022 # year to be used in the file name
max_size = 24.8 # megabytes
initial_quality = 100 # arbitrary units from 0-100
folder_01 = "kofi" # name of new folder_01
folder_02 = "socials" # name of new folder_02
img_max_dim = 860 #pixels, set the maximum size for the resized images for folder_02

## Create a function to compress images

In [ ]:
# function compress_image
## agrs: image_path, max_size, new_file_path 
## returns: original_size, new_file_path, quality, new_filesize
def compress_image(image_path, max_size, new_file_path):
    # initial check to avoid future errors
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"{image_path} does not exist")
    if not os.path.isfile(image_path):
        raise ValueError(f"{image_path} is not a file")
    if not image_path.endswith(('.jpeg', '.jpg')):
        raise ValueError(f"{image_path} is not a valid image file")
    try:
        image = Image.open(image_path)
    except Exception as e:
        raise ValueError(f"Unable to open {image_path}: {e}")
    original_size = round(os.path.getsize(image_path) / 1024 / 1024,3)
    quality = initial_quality # arg: quality in save function
    # compare the imgage size to the threshold to compress
    if original_size <= max_size:
        # if < then only copy and rename
        new_filesize = original_size
        shutil.copy2(image_path, new_file_path)
        return original_size,new_file_path, quality, new_filesize
    else:
        # if > then iterate by subtracting 1 to the quality variable that start at 100
        while original_size > max_size:  
            EXIF = image.getexif()
            image.save(new_file_path, optimize=True, quality=quality, exif= EXIF)
            new_filesize= round(os.path.getsize(new_file_path) / 1024 / 1024,3)
            quality -= 1
            if new_filesize <= max_size:
                break
        return original_size,new_file_path, quality, new_filesize

## Run function in the current folder

In [ ]:
folder_path = os.getcwd() # get current folder path
project_name = os.path.basename(folder_path) # get name of current folder == project folder
# check existance of folder_01, if does not exist then create new folder_01 
new_folder_path = os.path.join(folder_path, folder_01)
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

i = 1 # iterator for sequential naming
# create empty table to export later
table_data = [['original_filename', 'new_filename','original_filesize', 'new_filesize', 'compression']]
# interate all the images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpeg', '.jpg')):
        file_path = os.path.join(folder_path, filename)
        # create new file name `YYY_ParentFolder_0`
        new_filename = str(year) + "_" + project_name + '_0' + str(i) + '.' + filename.split('.')[1]
        i += 1 
        new_file_path = os.path.join(new_folder_path, new_filename)
        original_size, new_file_path, quality, new_filesize = compress_image(file_path, max_size, new_file_path)
        table_data.append([filename, original_size, new_filename, new_filesize, quality])
# create a new path variable with path from folder_01 to be used later
path_processed_images = new_folder_path

### Table with modified images

In [ ]:
# export table to csv file
with open(os.path.join(folder_path, project_name + '_exported_images.csv'), 'w') as f:
    writer = csv.writer(f)
    writer.writerows(table_data)
# create pandas DataFrame from table data and display formatted table
df = pd.DataFrame(table_data[1:], columns=table_data[0])
display(df.style.hide_index())

## Create thumbnail/social media sized images 

In [ ]:
# Create the new folder_02 if it doesn't already exist
new_folder_path = os.path.join(folder_path, folder_02)
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

# Loop through the image files in the original folder
for filename in os.listdir(path_processed_images):
    if filename.endswith(('.jpeg', '.jpg', '.png')):
        file_path = os.path.join(path_processed_images, filename)
        
        # Open the image file and calculate the new size
        with Image.open(file_path) as img:
            width, height = img.size
            ratio = min(img_max_dim / width, img_max_dim / height)
            new_size = (int(width * ratio), int(height * ratio))
            EXIF = img.getexif()
            # Resize the image while maintaining its aspect ratio
            img = img.resize(new_size, Image.ANTIALIAS)
            
            # Save the resized image to the new folder
            new_file_path = os.path.join(new_folder_path, filename)
            img.save(new_file_path,exif=EXIF)

All done. 